# Team Raj Datathon 2025

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split

In [13]:
# reading in data
visitation_df=pd.read_csv('data/visitation_data.csv')
climate_df=pd.read_csv('data/climate_data.csv')

In [ ]:
# cleaning climate data

# trim pre-2014 data
climate_filtered_df = climate_df[climate_df['Year'] >= 2014]

# get ONLY snow seasons for each year
climate_filtered_df = climate_filtered_df[
    ((climate_filtered_df['Month'] == 6) & (climate_filtered_df['Day'] >= 9)) |
    ((climate_filtered_df['Month'].isin([7,8]   ))) | 
    ((climate_filtered_df['Month'] == 9) & (climate_filtered_df['Day'] <= 21))
    ]

# datetime processing
climate_filtered_df['Date'] = pd.to_datetime(
    dict(year=2000, month=climate_filtered_df['Month'], day=climate_filtered_df['Day'])
)

# adding a week column for each date
start_date = pd.Timestamp(year=2000, month=6, day=9)
climate_filtered_df['DaysSinceJune9'] = (climate_filtered_df['Date'] - start_date).dt.days
climate_filtered_df['Week'] = (climate_filtered_df['DaysSinceJune9'] // 7) + 1
climate_filtered_df.drop(columns='DaysSinceJune9', inplace=True)
climate_filtered_df.drop(columns='Date', inplace=True)

# averaging over each week
week_av_clim_df = climate_filtered_df.groupby(
    ['Bureau of Meteorology station number', 'Year', 'Week']
    ) [[
        'Maximum temperature (Degree C)',
        'Minimum temperature (Degree C)',
        'Rainfall amount (millimetres)'
    ]].mean().reset_index()

# deal with remaining NaN that weren't removed by averaging over each week
# imputing on median (reduces impact of outlier weather conditions)  

cols_to_fill = [
    'Maximum temperature (Degree C)',
    'Minimum temperature (Degree C)',
    'Rainfall amount (millimetres)'
]

station_week_clim_avg = week_av_clim_df.groupby(['Bureau of Meteorology station number', 'Week'])[cols_to_fill].transform('median')
week_av_clim_df[cols_to_fill] = week_av_clim_df[cols_to_fill].fillna(station_week_clim_avg)

1210
